In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, KFold
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data=pd.read_csv('Data_Set/Perovskite_database_content_all_data.csv')

In [ ]:
data.shape

In [ ]:
data['Perovskite_thickness']

In [ ]:
def replace(x):
    if x in ['Unknown','none','unknown','None']:
        return np.nan
    elif 'Unknown' in str(x):
        return np.nan
    elif 'none' in str(x):
        return np.nan
    elif 'None' in str(x):
        return np.nan
    elif 'nan' in str(x):
        return np.nan
    else:
        return x

In [ ]:
data=data.applymap(lambda x :replace(x))

In [ ]:
data.shape

In [ ]:
data=data.applymap(lambda x: x.replace(' ','') if isinstance(x,str) else x)

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
data.dropna(axis='columns', how ='all',inplace=True)

In [ ]:
data.dropna(axis='rows', how ='all',inplace=True)

In [ ]:
data.shape

In [ ]:
data.dropna(subset=['Perovskite_composition_a_ions',
 'Perovskite_composition_a_ions_coefficients',
 'Perovskite_composition_b_ions',
 'Perovskite_composition_b_ions_coefficients',
 'Perovskite_composition_c_ions',
 'Perovskite_composition_c_ions_coefficients','Perovskite_dimension_3D','Perovskite_thickness','Perovskite_band_gap'],inplace=True)

In [ ]:
data.shape

In [ ]:
perovskite_comp=['Perovskite_composition_a_ions',
 'Perovskite_composition_a_ions_coefficients',
 'Perovskite_composition_b_ions',
 'Perovskite_composition_b_ions_coefficients',
 'Perovskite_composition_c_ions',
 'Perovskite_composition_c_ions_coefficients','Perovskite_dimension_3D','Perovskite_thickness','Perovskite_band_gap']

In [ ]:
perovskite_comp

In [ ]:
perovskite_data=data[perovskite_comp]

In [ ]:
perovskite_data.shape

In [ ]:
perovskite_data=perovskite_data.dropna()

In [ ]:
perovskite_data['Layered']=perovskite_data['Perovskite_composition_a_ions'].apply(lambda x: True if len(str(x).split('|'))>1 else False)

In [ ]:
perovskite_data=perovskite_data[~perovskite_data['Layered']]

In [ ]:
perovskite_data=perovskite_data[perovskite_data['Perovskite_dimension_3D']]

In [ ]:
same_dict={'Blowinghotair':'Blow-drying',
           'Comersial':'Commercial',
           'Electrohemicalanodization':'Electrochemicalanodization',
           'Electropolymerisation':'Electropolymerization',
           'Evaoration': 'Evaporation',
           'Gasreaction':'Gas-reaction',
           'Hydorthermal':'Hydorthermal',
           'Hydrolys':'Hydrolysis',
           'Inkjetprinting' :'Inkjet-Printed',
           'RFMagnetronSputtering': 'RFMagnetronsputtering',
           'RFmagnetronsputtering': 'RFMagnetronsputtering',
           'SILARmethod':'SILAR',
           'Screenprintnig':'Screenprinting',
           'Sputering':'Sputtering',
           'Ultrasonicspraypyrolysis':'Ultrasonicspray'}

In [ ]:
perovskite_data.drop_duplicates(inplace=True)
perovskite_data.dropna(inplace=True)

In [ ]:
a_ions=perovskite_data['Perovskite_composition_a_ions'].str.split(';').str.join(' ').str.split(expand=True).apply(pd.Series.value_counts).sum(axis=1)

In [ ]:
a_ions=a_ions[a_ions>200].index.tolist()

In [ ]:
b_ions=perovskite_data['Perovskite_composition_b_ions'].str.split(';').str.join(' ').str.split(expand=True).apply(pd.Series.value_counts).sum(axis=1)

In [ ]:
b_ions=b_ions[b_ions>=100].index.tolist()

In [ ]:
c_ions=perovskite_data['Perovskite_composition_c_ions'].str.split(';').str.join(' ').str.split(expand=True).apply(pd.Series.value_counts).sum(axis=1)

In [ ]:
c_ions=c_ions[c_ions>200].index.tolist()

In [ ]:
ions_token=list(set(a_ions+b_ions+c_ions))

In [ ]:
perovskite_data.drop_duplicates(subset=['Perovskite_composition_a_ions','Perovskite_composition_b_ions','Perovskite_composition_c_ions','Perovskite_thickness'],inplace=True)

In [ ]:
perovskite_data['Perovskite_thickness']=perovskite_data['Perovskite_thickness'].apply(lambda x: [float(x)/1000])

In [ ]:
features=['Cs', 'FA', 'MA','Pb', 'Sn','Br', 'I','Thickness']

In [ ]:
perovskite_data=perovskite_data[perovskite_data['Perovskite_composition_a_ions'].apply(lambda x : all([True if y in ions_token else False for y in x.split(';') ]))]      

In [ ]:
perovskite_data=perovskite_data[perovskite_data['Perovskite_composition_b_ions'].apply(lambda x : all([True if y in ions_token else False for y in x.split(';') ]))]        

In [ ]:
perovskite_data=perovskite_data[perovskite_data['Perovskite_composition_c_ions'].apply(lambda x : all([True if y in ions_token else False for y in x.split(';') ]))]        

In [ ]:
def ions_one_hot(x,c,tokens):
    if 'x' in c:
        return np.nan
    temp_dict={a:b for (a,b) in zip(x.split(';'),c.split(';'))}
    return [temp_dict[x] if x in temp_dict.keys() else 0 for x in tokens]

In [ ]:
perovskite_data['Perovskite_composition_a_ions']=perovskite_data[['Perovskite_composition_a_ions','Perovskite_composition_a_ions_coefficients']].apply(lambda x :ions_one_hot(*x,a_ions),axis=1)

In [ ]:
perovskite_data['Perovskite_composition_b_ions']=perovskite_data[['Perovskite_composition_b_ions','Perovskite_composition_b_ions_coefficients']].apply(lambda x :ions_one_hot(*x,b_ions),axis=1)

In [ ]:
perovskite_data['Perovskite_composition_c_ions']=perovskite_data[['Perovskite_composition_c_ions','Perovskite_composition_c_ions_coefficients']].apply(lambda x :ions_one_hot(*x,c_ions),axis=1)

In [ ]:
perovskite_data

In [ ]:
perovskite_data.dropna(inplace=True)

In [ ]:
for cols in ['Perovskite_composition_a_ions',
       'Perovskite_composition_b_ions',
       'Perovskite_composition_c_ions']:
    maximum=perovskite_data[cols].map(len).max()
    perovskite_data[cols]=perovskite_data[cols].apply(lambda x: x+[0]*(maximum-len(x)))

In [ ]:
perovskite_data['X_train']=perovskite_data[['Perovskite_composition_a_ions',
       'Perovskite_composition_b_ions',
       'Perovskite_composition_c_ions','Perovskite_thickness']].sum(axis=1)

In [ ]:
perovskite_data

In [ ]:
train=np.asanyarray(perovskite_data['X_train'].tolist(),dtype=float)

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="5"

In [ ]:
label=np.asanyarray(perovskite_data['Perovskite_band_gap'].tolist(),dtype=float)

In [ ]:
from sklearn.utils import shuffle

train , label = shuffle(train,label, random_state=12)

In [ ]:
# save to csv file
from numpy import savetxt
savetxt('Data_Set/train.csv', train, delimiter=',')
savetxt('Data_Set/label.csv', label, delimiter=',')

In [ ]:
from numpy import loadtxt
# load array
train = loadtxt('Data_Set/train.csv', delimiter=',')
label = loadtxt('Data_Set/label.csv', delimiter=',')

In [ ]:
train.shape

In [ ]:
import matplotlib.pyplot as plt

# Create the data
data = [2, 3, 3, 4, 4, 4, 5, 5, 6, 6, 6, 6, 7, 7, 7, 8, 8, 9, 10]

# Plot the histogram
plt.hist(train, bins=8)

# Add a title and labels
plt.title("Histogram of Data")
plt.xlabel("Value")
plt.ylabel("Frequency")

# Show the plot
plt.show()